In [1]:
import os
import cv2
import torch
import numpy as np
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import MetadataCatalog
from torchvision.transforms.functional import to_tensor
import torch.nn.functional as F
from datasets import load_dataset
import pickle

In [2]:
def setup_detectron2_model(config_file, model_weights, device):
    cfg = get_cfg()
    cfg.merge_from_file(config_file)
    cfg.MODEL.RETINANET.SCORE_THRESH_TEST = 0.5
    cfg.MODEL.WEIGHTS = model_weights
    cfg.MODEL.DEVICE = device

    model = build_model(cfg)
    model.eval()
    DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)
    return model

In [3]:
cifar100_data = load_dataset("cifar100")

Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/datasets/features/image.py:325: UserWarning: Downcasting array dtype uint8 to uint8 to be compatible with 'Pillow'
  warnings.warn(f"Downcasting array dtype {dtype} to {dest_dtype} to be compatible with 'Pillow'")


Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset cifar100 downloaded and prepared to /root/.cache/huggingface/datasets/cifar100/cifar100/1.0.0/f365c8b725c23e8f0f8d725c3641234d9331cd2f62919d1381d1baa5b3ba3142. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
train_data = cifar100_data["train"]
test_data = cifar100_data["test"]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
config_file = "faster_rcnn_R_50_FPN_3x.yaml"
model_weights = "detectron2://COCO-Detection/faster_rcnn_R_50_FPN_3x/137849458/model_final_280758.pkl"
model = setup_detectron2_model(config_file, model_weights, device)

In [6]:
def extract_roi_features(image, model):
    image = to_tensor(image)  # Convert image to a PyTorch tensor
    image = image.to(model.device)  # Move image to the same device as the model
    image = (image * 255).to(torch.uint8)  # Convert back to the original range (0-255)

    with torch.no_grad():
        input_image = model.preprocess_image([{"image": image, "height": image.shape[-2], "width": image.shape[-1]}])
        features = model.backbone(input_image.tensor)
        proposals, _ = model.proposal_generator(input_image, features)
        box_features = model.roi_heads.box_pooler(
            [features[f] for f in features if f!='p6'],
            [p.proposal_boxes for p in proposals]
        )
        box_features = model.roi_heads.box_head(box_features) ## FINAL Features
        
    return box_features.cpu()

In [12]:
print("Generating " + str(len(train_data)) + " embeddings")

visual_embeddings = []
fine_labels = []
coarse_labels = []

for idx, img_data in enumerate(train_data):
    if idx % 1000 == 0:
        print("On idx " + str(idx))
    image = np.array(img_data["img"])
    embedding = extract_roi_features(image, model)
    visual_embeddings.append(embedding)
    fine_labels.append(img_data["fine_label"])
    coarse_labels.append(img_data["coarse_label"])


Generating 50000 embeddings
On idx 0
On idx 1000
On idx 2000
On idx 3000
On idx 4000
On idx 5000
On idx 6000
On idx 7000
On idx 8000
On idx 9000
On idx 10000
On idx 11000
On idx 12000
On idx 13000
On idx 14000
On idx 15000
On idx 16000
On idx 17000
On idx 18000
On idx 19000
On idx 20000
On idx 21000
On idx 22000
On idx 23000
On idx 24000
On idx 25000
On idx 26000
On idx 27000
On idx 28000
On idx 29000
On idx 30000
On idx 31000
On idx 32000
On idx 33000
On idx 34000
On idx 35000
On idx 36000
On idx 37000
On idx 38000
On idx 39000
On idx 40000
On idx 41000
On idx 42000
On idx 43000
On idx 44000
On idx 45000
On idx 46000
On idx 47000
On idx 48000
On idx 49000


In [13]:
cifar100_train_embeddings = {
    "embeddings": visual_embeddings,
    "fine_labels": fine_labels,
    "coarse_labels": coarse_labels
}

with open("cifar100-train-embeddings.pkl", "wb") as f:
    pickle.dump(cifar100_train_embeddings, f)

In [9]:
print("Generating " + str(len(test_data)) + " embeddings")

visual_embeddings = []
fine_labels = []
coarse_labels = []

for idx, img_data in enumerate(test_data):
    if idx % 1000 == 0:
        print("On idx " + str(idx))
    image = np.array(img_data["img"])
    embedding = extract_roi_features(image, model)
    visual_embeddings.append(embedding)
    fine_labels.append(img_data["fine_label"])
    coarse_labels.append(img_data["coarse_label"])


Generating 10000 embeddings
On idx 0
On idx 1000
On idx 2000
On idx 3000
On idx 4000
On idx 5000
On idx 6000
On idx 7000
On idx 8000
On idx 9000


In [10]:
cifar100_test_embeddings = {
    "embeddings": visual_embeddings,
    "fine_labels": fine_labels,
    "coarse_labels": coarse_labels
}

with open("cifar100-test-embeddings.pkl", "wb") as f:
    pickle.dump(cifar100_test_embeddings, f)